# libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time
import scipy.optimize as optimize
import lightgbm as lgb
from bs4 import BeautifulSoup


from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import rankdata

from collections import defaultdict
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from tqdm import tqdm

import time
import re 
import scipy
from scipy import sparse
import gc 
from IPython.display import display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

In [2]:
"""
fold type =1 用kfold方式分隔数据集 
fold type=2  用正负采样方式采样数据集
实验证明 2比1好 比例在1:1 到1.5 之间 cv有提高 lb没有变化
后面代码中带k 的为kfold 带s的为sample 采样
fold type2 慎用，很久没用可能兼容bug多
"""
Fold_type=2
is_test=False
translate_aug=False
fast_vec=True
#记得改kdict 里面的tranlate的值


In [3]:
fold_num_k=5
fold_num_s=1

fast_factor=1

In [4]:
"""
如果需要增加数据集 检查 data_names, 是否clean 检查clean data，create fold 处，name2dict处
"""

'\n如果需要增加数据集 检查 data_names, 是否clean 检查clean data，create fold 处，name2dict处\n'

In [5]:
# data_names=["jc_","juc_","rud_","jcc_"]
# model_choice=["ridge","gbm"]
# factor=[0.5,0.5]

"""
data_names 使用的数据集 可用的包括 (数据名称后面的_不能少):
jc_ :代表 第一届数据 
rud_ rud 数据
jcc1_ 对jc使用第一种clean方式
jcc2_ 对jc使用第二种clean方式

juc_ 第二届比赛数据

jc_fr_ 将第一届比赛数据进行反译 先由en-》french-》en 
jc_de_ 
jc_es_

最好的结果是由jc_ rud juc1 产生 0.870 （0.873 是由于失误出现的，不可靠） 
数据加权结果是由 各个数据集归一化后简单求和，后期可以再调整。 
"""

# data_names=["jc_","rud_","jcc1_","jcc2_","jc_fr_","jc_de_","jc_es_"]
# data_names=["jc_","rud_"]
data_names=["jc_","rud_","jcc1_","jcc2_"]

jc_num,jc_rate,rud_rate=3,4,1
factor_data=[jc_rate/jc_num,rud_rate,jc_rate/jc_num,jc_rate/jc_num,jc_rate/jc_num,jc_rate/jc_num,jc_rate/jc_num]

#这里是用于翻译增强 可以不用管
translate_data=["jc_s_"]
translate_language=["fr_text","de_text","es_text"]
clean_data={"jcc1_s_":1,"jcc2_s_":2}

#这里是选择模型 可选参数 包括 ridge 和gbm factor用于指定相应权值 ，gbm重复试验的结果看效果不好，选择放弃
model_choice=["ridge"]
factor=[1]

In [6]:
system_path=r"../input"
model_all=["ridge","gbm"]
for model_name in model_all:
    if model_name not in os.listdir():
        os.makedirs(f"{model_name}")
out_path=r"./"

# Data

In [7]:


#第一届 jigsaw比赛 数据（challenge） Toxic Comment Classification Challenge

jc_path=os.path.join(system_path,"jigsaw-toxic-comment-classification-challenge")
jc_trans_path=os.path.join(system_path,"jc-trans")
#ruddit 数据\
run_path=os.path.join(system_path,"ruddit-jigsaw-dataset/Dataset")
#第二届 jigsaw比赛 对少数人群不歧视
juc_path=os.path.join(system_path,"jigsaw-unintended-bias-in-toxicity-classification")

#本次比赛数据 作为val
jts_path=os.path.join(system_path,"jigsaw-toxic-severity-rating")

# #数据抽样存储路径
gbm_save_path=os.path.join(out_path,"gbm")
ridge_save_path=os.path.join(out_path,"ridge")

In [8]:
#验证集和测试集
df_val = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))

df_test = pd.read_csv(os.path.join(jts_path,"comments_to_score.csv"))

导入数据

In [9]:
#第一届比赛数据 以0/1为分值 
features = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

jc_train_df = pd.read_csv(os.path.join(jc_path,"train.csv"))
jc_test_df = pd.read_csv(os.path.join(jc_path,"test.csv"))
temp_df = pd.read_csv(os.path.join(jc_path,"test_labels.csv"))

jc_test_df = jc_test_df.merge ( temp_df, on ="id")
#drop test data not used for scoring
jc_test_df = jc_test_df.query ("toxic != -1")
jc_test_df=jc_test_df[jc_test_df[features].sum(axis=1)>0]

jc_df = jc_train_df.append ( jc_test_df ) 

# jc_df=jc_train_df

# 将代表有毒行为的筛选出来
jc_df["toxic_subtype_sum"]=jc_df[features].sum(axis=1)
jc_df["toxic_behaviour"]=jc_df["toxic_subtype_sum"].map(lambda x: x > 0)

tot_toxic_behaviour = jc_df["toxic_behaviour"].sum()
print(f'comments with toxic behaviour:{tot_toxic_behaviour}')
jc_df=jc_df.reset_index(drop=True)

comments with toxic behaviour:22468


In [10]:
#第一届比赛 数据预处理
# toxic = 1.0
# severe_toxic = 2.0
# obscene = 1.0
# threat = 1.0
# insult = 1.0
# identity_hate = 2.0
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}
PSEUDO_LABEL_WEIGHT = 0.033
toxic_labels=[k for k in cat_mtpl.keys()]
def create_train (df):
    
    for category in cat_mtpl:
        df[category] = df[category] * cat_mtpl[category]
    df['y'] = df.loc[:, toxic_labels].sum(axis=1)
    
    
    df = df[["id",'comment_text', 'y', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].rename(columns={'comment_text': 'text'})
    return df
        
jc_df = create_train (jc_df)
jc_df.shape

(165814, 9)

In [11]:
# pseudo = pd.read_csv('../input/toxic-public-dataframes/old_pseudo_label.csv')
# comment_to_pseudo_label = pseudo.set_index('comment_text').to_dict()['score']
# jc_df['soft_pseudo_label_score'] = jc_df.text.map(comment_to_pseudo_label)

# jc_df.y = jc_df.y + PSEUDO_LABEL_WEIGHT*jc_df.soft_pseudo_label_score


In [12]:
#jc 翻译数据处理
jc_trans=pd.read_csv(os.path.join(jc_trans_path,"jc_trans.csv"))
jc_trans.set_index(["Unnamed: 0"], inplace=True)
def translation_df(jc_trans,language_text):
    df=pd.merge(jc_df,jc_trans[["id",language_text]],on="id",how="left")
    df["text"]=df.apply(lambda row:row["text"] if pd.isna(row[language_text]) else row[language_text],axis=1)
    df.drop([language_text],axis=1,inplace=True)
    return df
jc_fr_df=translation_df(jc_trans,"fr_text")
jc_es_df=translation_df(jc_trans,"es_text")
jc_de_df=translation_df(jc_trans,"de_text")

In [13]:
# 第二届比赛 以打分形式0~1
features = ["toxicity","severe_toxicity","obscene","insult","identity_attack", "sexual_explicit"]
cols = ['id', 'comment_text', 'toxicity', 'severe_toxicity', 'obscene', 'threat','insult', 'identity_attack', 'sexual_explicit', 'toxicity_annotator_count']

juc_df = pd.read_csv(os.path.join(juc_path,"all_data.csv"))

juc_df = juc_df.query ("toxicity_annotator_count > 5")
print(f"juc_df:{juc_df.shape}")
juc_df['y'] = juc_df[[ 'severe_toxicity', 'obscene', 'sexual_explicit','identity_attack', 'insult', 'threat']].sum(axis=1)

#毒性<0.5 按标注 否则算所有有毒行为的标注和
juc_df['y'] = juc_df.apply(lambda row: row["toxicity"] if row["toxicity"] <= 1 else row["y"] , axis=1)
juc_df = juc_df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
# min_len = (juc_df['y'] > 0.5).sum()


juc_df:(601044, 46)


In [14]:
#ruddit 数据
rud_df = pd.read_csv(os.path.join(run_path,"ruddit_with_text.csv"))

print(f"rud_df:{rud_df.shape}")
rud_df['y'] = rud_df['offensiveness_score'].map(lambda x: 0.0 if x <=0 else x)
# rud_df['y'] = rud_df['offensiveness_score']

rud_df = rud_df[['txt', 'y']].rename(columns={'txt': 'text'})
dele_flag="[deleted]"
rud_df=rud_df.query("text!=@dele_flag")

rud_df:(5838, 5)


In [15]:
#clean data
def clean1(text):

    # Clean some punctutations
    text=re.sub(r'\n', r' \n ',text)
    text=re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3',text)
    
    # Replace repeating characters more than 3 times to length of 3
    text=re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1',text)
    
    # Add space around repeating characters
    text=re.sub(r'([*!?\']+)',r' \1 ',text)
    
    # patterns with repeating characters 
    text=re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1',text)
    text=re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1',text)
    text=re.sub(r'[ ]{2,}',' ',text)
    text = text.strip()
    
    return text


def clean2(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text
def clean3(text):
    '''
    1+2
    '''
    text=re.sub(r'\n', r' \n ',text)
    text=re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3',text)
    
    # Replace repeating characters more than 3 times to length of 3
    text=re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1',text)
    
    # Add space around repeating characters
    text=re.sub(r'([*!?\']+)',r' \1 ',text)
    
    # patterns with repeating characters 
    text=re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1',text)
    text=re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1',text)
    text=re.sub(r'[ ]{2,}',' ',text)
    text = text.strip()
    
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text


In [16]:
# test_clean_df = pd.DataFrame({"text":
#                               ["heyy\n\nkkdsfj",
#                                "hi   how/are/you ???",
#                                "hey?????",
#                                "hey????? 18.98.333.20 18.98.",
#                                "noooo!!!!!!!!!   comeone !! ",
#                               "cooooooooool     brooooooooooo  coool brooo",
#                               "naaaahhhhhhh"]})
# display(test_clean_df)

In [17]:
tqdm.pandas()
# clean 数据
def clean_df(df,clean_type):
    clean_df=df.copy()
    
    if clean_type==1:
        
        clean_df['text']=clean_df['text'].progress_apply(clean1)
    elif clean_type==2:
        
        clean_df['text']=clean_df['text'].progress_apply(clean2)
    elif clean_type==3:
        clean_df['text']=clean_df['text'].progress_apply(clean3)
    return clean_df
jcc1_df=clean_df(jc_df,1)
jcc2_df=clean_df(jc_df,2)
# jcc3_df=clean_df(jc_df,3)

100%|██████████| 165814/165814 [01:01<00:00, 2686.41it/s]


# 消重

In [18]:
# 消除jc中重叠数据
df_jc_val = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))
print(df_jc_val.shape)


# Find cases already present in toxic data

df_jc_val = pd.merge(df_jc_val, jc_df.loc[:,['text']], 
                  left_on = 'less_toxic', 
                  right_on = 'text', how='left')

df_jc_val = pd.merge(df_jc_val, jc_df.loc[:,['text']], 
                  left_on = 'more_toxic', 
                  right_on = 'text', how='left')

# Removing those cases
# df_jc_val = df_jc_val[(~df_jc_val.text_x.isna()) | (~df_jc_val.text_y.isna())][['worker', 'less_toxic', 'more_toxic']]
df_jc_val = df_jc_val[(df_jc_val.text_x.isna()) & (df_jc_val.text_y.isna())][['worker', 'less_toxic', 'more_toxic']]

df_jc_val.shape

(30108, 3)


(6489, 3)

### aug_trans

In [19]:
if translate_aug==True:
    jc_trans=pd.read_csv(os.path.join(jc_trans_path,"jc_trans.csv"))
    jc_trans.set_index(["Unnamed: 0"], inplace=True)

    jc_df=pd.merge(jc_df,jc_trans[["id","fr_text"]],on="id",how="left")
    jc_df=pd.merge(jc_df,jc_trans[["id","es_text"]],on="id",how="left")
    jc_df=pd.merge(jc_df,jc_trans[["id","de_text"]],on="id",how="left")

## 分值显示

In [20]:
def show_data(train_df):
    printed = []
    for i in sorted(train_df.y.unique()):
        n = np.round(i, 2) 
        if n in printed:
            continue
        printed.append(n)
        print(f"{len(printed):<3}: {i:.5f}\t{repr(np.random.choice(train_df[train_df.y==i]['text']))[:300]}")

# create fold

In [21]:
def create_fold_kfold(df,FOLDS=5,select_num=0.5,frac_1=1.5,balance=False):
    #select num challenge:1 unbias:0.5  

    min_len = (df['y'] >= select_num).sum()
    if balance==False:
        #采样负样本
        df_y0_undersample = df[df['y'] <select_num].sample(n=int(min_len*frac_1),random_state=201)
        df = pd.concat([df[-(df['y'] <select_num)], df_y0_undersample])
        df=df.reset_index(drop=True)
    y=df["y"].values
    x=df["text"]
    stratified = np.around (y)
    
    kf = StratifiedKFold(n_splits=FOLDS,shuffle=True,random_state=123)
    df_dict={}
    for fold,(train_index,val_index) in enumerate(kf.split(x,stratified)):
        df_train=df.iloc[train_index]
        df_dict[fold]=df_train
    
    return df_dict

def create_fold_sample(df,n_folds=3,frac_1=0.8,frac_1_factor=1.5,select_num=0,balance=False,translate=False):
    df_dict={}
    #正样本 大于等于select_num
    select_list=-(df['y'] <select_num)
    min_len=select_list.sum()
    if translate==True:
        min_len=4*min_len
    for fld in range(n_folds):
        if balance==False:
#             df_y0_undersample=df[df.y<select_num].sample(n=int(min_len*frac_1*frac_1_factor), random_state = 10*(fld+1))
            df_y0_undersample=df[df.y<select_num].sample(n=int(min_len*frac_1*frac_1_factor), random_state = 201)

            tmp_df = pd.concat([df[select_list].sample(frac=frac_1, random_state = 10*(fld+1)),df_y0_undersample ])
        else:
            tmp_df = df.sample(frac=frac_1, random_state = 10*(fld+1))
            
        df_dict[fld]=tmp_df

    return df_dict


In [22]:
if is_test==True:
    jc_df=jc_df[0:400]
    juc_df=juc_df[0:400]
    rud_df=rud_df[0:400]
    jcc_df=jcc_df[0:400]

In [23]:
if is_test==True:
#     jc_df_kdict=create_fold_kfold(jc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=True)
#     juc_df_kdict=create_fold_kfold(juc_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     rud_df_kdict=create_fold_kfold(rud_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     jcc_df_kdict=create_fold_kfold(jcc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=True)

    jc_df_sdict=create_fold_sample(jc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.001,balance=True,translate=translate_aug)
    juc_df_sdict=create_fold_sample(juc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=True)
    rud_df_sdict=create_fold_sample(rud_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.5,balance=True)
    jcc1_df_sdict=create_fold_sample(jcc1_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.001,balance=True)
    jcc2_df_sdict=create_fold_sample(jcc2_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1.5,select_num=0.001,balance=True)
    
if is_test==False:
#     jc_df_kdict=create_fold_kfold(jc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=False)
#     juc_df_kdict=create_fold_kfold(juc_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=False)
#     rud_df_kdict=create_fold_kfold(rud_df,FOLDS=fold_num_k,select_num=0.5,frac_1=1.5,balance=True)
#     jcc_df_kdict=create_fold_kfold(jcc_df,FOLDS=fold_num_k,select_num=1,frac_1=1.5,balance=False)

    jc_df_sdict=create_fold_sample(jc_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False,translate=translate_aug)
    juc_df_sdict=create_fold_sample(juc_df,n_folds=fold_num_s,frac_1=0.8,frac_1_factor=1,select_num=0.5,balance=False)
    rud_df_sdict=create_fold_sample(rud_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1,select_num=0.5,balance=True)
    jcc1_df_sdict=create_fold_sample(jcc1_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False)
    jcc2_df_sdict=create_fold_sample(jcc2_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False)
    jc_fr_df_sdict=create_fold_sample(jc_fr_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False)
    jc_de_df_sdict=create_fold_sample(jc_de_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False)
    jc_es_df_sdict=create_fold_sample(jc_es_df,n_folds=fold_num_s,frac_1=1,frac_1_factor=1.5,select_num=0.001,balance=False)
    
    
#     jc_df_kdict[0].shape
#     juc_df_kdict[0].shape
#     rud_df_kdict[0].shape
#     jcc_df_sdict[0].shape

In [24]:
from scipy.sparse import hstack
from scipy import sparse
from gensim.models import KeyedVectors, FastText

fmodel = FastText.load('../input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin')
def splitter(text):
    tokens = []
    
    for word in text.split(' '):
        tokens.append(word)
    
    return tokens

def vectorizer(text):
    tokens = splitter(text)
    
    x = np.mean(fmodel.wv[tokens], axis = 0).reshape(1, -1)
    
    return np.squeeze(x,axis=0)   
def text2fasttextarray(text):
    X_fast=[]
    for t in text:
        X_fast.append(vectorizer(t))
    X_fast=np.matrix(X_fast)
    
    return X_fast


# ridge

In [25]:
import joblib
def ridge_cv(df_dic,n_folds,model_pre="jc_k_ridge_",df_val=df_val,clean_prm=0,translate=False):
    val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
    val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
    test_preds_arr = np.zeros((df_test.shape[0], n_folds))
    for fld in tqdm(range(n_folds)):
        df = df_dic[fld]
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5) )
        vec_pre=model_pre+"vec_"
        if translate!=True:
            text=df["text"]
            y=df["y"]
        else:
            trans_df=df.dropna(axis=0,subset = ["fr_text"])
            text=pd.concat([df["text"],trans_df["fr_text"],trans_df["es_text"],trans_df["de_text"]])
            y=pd.concat([df["y"],trans_df["y"],trans_df["y"],trans_df["y"]])
        X=vec.fit_transform(text)
        joblib.dump(vec,os.path.join(ridge_save_path,f'{vec_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        model=Ridge(alpha=0.5)
        model.fit(X,y)
        if fast_vec==True:
            X_f=text2fasttextarray(text)
            model_f=Ridge(alpha=0.5)
            model_f.fit(X_f,y)
        if clean_prm==1:
            X_less_toxic = df_val.apply(lambda row :clean1(row["less_toxic"]),axis=1)
            X_more_toxic = df_val.apply(lambda row :clean1(row["more_toxic"]),axis=1)
            X_test = df_test.apply(lambda row :clean1(row["text"]),axis=1)
    
        elif clean_prm==2:
            X_less_toxic = df_val.apply(lambda row :clean2(row["less_toxic"]),axis=1)
            X_more_toxic = df_val.apply(lambda row :clean2(row["more_toxic"]),axis=1)
            X_test = df_test.apply(lambda row :clean2(row["text"]),axis=1)
        elif clean_prm==3:
            X_less_toxic = df_val.apply(lambda row :clean3(row["less_toxic"]),axis=1)
            X_more_toxic = df_val.apply(lambda row :clean3(row["more_toxic"]),axis=1)
            X_test = df_test.apply(lambda row :clean3(row["text"]),axis=1)

        else:
            X_less_toxic = df_val['less_toxic']
            X_more_toxic = df_val['more_toxic']
            X_test = df_test['text']
        if fast_vec==True:
            X_less_toxic_f=text2fasttextarray(X_less_toxic)
            X_more_toxic_f=text2fasttextarray(X_more_toxic)
            X_test_f=text2fasttextarray(X_test)
            
            val_preds_arr1[:,fld]=model_f.predict(X_less_toxic_f)*fast_factor
            val_preds_arr2[:,fld]=model_f.predict(X_more_toxic_f)*fast_factor
            test_preds_arr[:,fld]=model_f.predict(X_test_f)*fast_factor
            joblib.dump(model,os.path.join(ridge_save_path,f'{model_pre}_f_{fld}.pkl')) #保存模型 文件后缀为.pkl
            
        X_less_toxic = vec.transform(X_less_toxic)
        X_more_toxic = vec.transform(X_more_toxic)
        X_test = vec.transform(X_test)
        
            
        val_preds_arr1[:,fld] += model.predict(X_less_toxic)
        val_preds_arr2[:,fld] += model.predict(X_more_toxic)

        test_preds_arr[:,fld] += model.predict(X_test)
            
        joblib.dump(model,os.path.join(ridge_save_path,f'{model_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        del model,vec
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')        
    return p1,p2,pv

# lightgbm

In [26]:
#verbose =-1 忽略警告
depth=1000
params = {'task':'train',
    "device" : "cpu",
    'boosting_type':'gbdt',
    "max_depth":7,
    "num_leaves":80,
    "bagging_fraction":0.8,
          "bagging_freq":5,
    'objective':'regression',
    "learning_rate":0.05,
    "max_bin":63,
    "random_state":2021,
    'verbose':-1, 
    "train_metric":False,
    "force_col_wise":False}


def lightgbm_cv(df_dic,n_folds,model_pre="jc_k_gbm_",df_val=df_val,clean_prm=False,translate=False):
    ####lightgbm_cv 在aug情况下有bug
    val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
    val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
    test_preds_arr = np.zeros((df_test.shape[0], n_folds))

    for fld in tqdm(range(n_folds)):
        df = df_dic[fld]
        vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(3, 5) )
        if translate!=True:
            text=df["text"]
            y=df["y"]
        else:
            trans_df=df.dropna(axis=0,subset = ["fr_text"])
            text=pd.concat([df["text"],trans_df["fr_text"],trans_df["es_text"],trans_df["de_text"]])
            y=pd.concat([df["y"],trans_df["y"],trans_df["y"],trans_df["y"]])

        X=vec.fit_transform(text)
        vec_pre=model_pre+"vec_"
        joblib.dump(vec,os.path.join(gbm_save_path,f'{vec_pre}{fld}.pkl')) #保存模型 文件后缀为.pkl
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.1,random_state = 0)
        lgb_train = lgb.Dataset(X_train,y_train)
        lgb_eval = lgb.Dataset(X_val,y_val,reference=lgb_train)
        
        gbm = lgb.train(params,lgb_train,
            num_boost_round=300,
            valid_sets=lgb_eval,
            early_stopping_rounds=50
            ) 
        
        if clean_prm==True:
            X_less_toxic = vec.transform(clean(df_val,'less_toxic')['less_toxic'])
            X_more_toxic = vec.transform(clean(df_val,'more_toxic')['more_toxic'])
            X_test = vec.transform(clean(df_test,'text')['text'])
        else:
            X_less_toxic = vec.transform(df_val['less_toxic'])
            X_more_toxic = vec.transform(df_val['more_toxic'])
            X_test = vec.transform(df_test['text'])
    
        val_preds_arr1[:,fld] = gbm.predict(X_less_toxic,num_iteration=gbm.best_iteration)
        val_preds_arr2[:,fld] = gbm.predict(X_more_toxic,num_iteration=gbm.best_iteration)

        test_preds_arr[:,fld] =gbm.predict(X_test,num_iteration=gbm.best_iteration)
        
        gbm.save_model(os.path.join(gbm_save_path,f'{model_pre}{fld}.txt'))
        # 模型加载
#         gbm = lgb.Booster(model_file='model.txt')
        # 模型预测
#         y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        del gbm,vec
    p1=val_preds_arr1.mean(axis=1)
    p2=val_preds_arr2.mean(axis=1)
    pv=test_preds_arr.mean(axis=1)
    print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')        
    return p1,p2,pv

In [27]:
p1=defaultdict()
p2=defaultdict()
pv=defaultdict()

val_data=df_val

func_dict={"ridge":ridge_cv,"gbm":lightgbm_cv}
# func_dict.get(x)

if Fold_type==1:
#     pre_names=["jc_k_","juc_k_","rud_k_","jcc_k_"]
    pre_names=[ data_name+"k_" for data_name in data_names]
    name2dict={"jc_k_":jc_df_kdict,"juc_k_":juc_df_kdict,"rud_k_":rud_df_kdict,
               "jcc1_k_":jcc1_df_kdict,"jcc2_k_":jcc2_df_kdict}
    fold_num=fold_num_k
elif Fold_type==2:
    pre_names=[ data_name+"s_" for data_name in data_names]
    name2dict={"jc_s_":jc_df_sdict,"juc_s_":juc_df_sdict,"rud_s_":rud_df_sdict,
               "jcc1_s_":jcc1_df_sdict,"jcc2_s_":jcc2_df_sdict,
               "jc_fr_s_":jc_fr_df_sdict,"jc_de_s_":jc_de_df_sdict,"jc_es_s_":jc_es_df_sdict}
    fold_num=fold_num_s

p1_ensenmble = np.zeros((val_data.shape[0]))
p2_ensenmble = np.zeros((val_data.shape[0]))
score=np.zeros((df_test.shape[0]))

for d_index,pre_name in enumerate(pre_names):
    ###model_pre_ridge:jc_s_ridge_ pre_name:jc_s_ model:jc_s_ridge_{fold} vec:jc_s_ridge_vec_{fold}
    #pre_name jc_s_ model_name jc_s_ridge_
    clean_prm=False
    translate=False
    
    p1[pre_name],p2[pre_name]=np.zeros((val_data.shape[0])),np.zeros((val_data.shape[0]))
    pv[pre_name]=np.zeros((df_test.shape[0]))
    if pre_name in clean_data.keys():
        clean_prm=clean_data[pre_name]
    if any([ trans_name in pre_name for trans_name in translate_data]) and translate_aug==True:
        translate=True

    for index,model_name in enumerate(model_choice):
        cv_func=func_dict.get(model_name)
        model_pre=pre_name+model_name+"_"
        p1[model_pre],p2[model_pre],pv[model_pre]=cv_func(name2dict[pre_name],n_folds=fold_num,df_val=val_data,
                                                        model_pre=model_pre,clean_prm=clean_prm,translate=translate)

        p1[pre_name]= p1[pre_name]+ p1[model_pre]*factor[index]
        p2[pre_name]= p2[pre_name]+ p2[model_pre]*factor[index]
        pv[pre_name]= pv[pre_name]+ pv[model_pre]*factor[index]

    kmax=max(p1[pre_name].max(),p2[pre_name].max())
    p1_ensenmble=p1_ensenmble+factor_data[d_index]*p1[pre_name]/kmax
    p2_ensenmble=p2_ensenmble+factor_data[d_index]*p2[pre_name]/kmax
    score=score+factor_data[d_index]*pv[pre_name]/kmax

print(f' Validation Accuracy is { np.round((p1_ensenmble < p2_ensenmble).mean() * 100,4)}') 

    
df_test['score'] = rankdata(score, method='ordinal')


100%|██████████| 1/1 [04:58<00:00, 298.56s/it]


Validation Accuracy is 67.52


100%|██████████| 1/1 [02:42<00:00, 162.37s/it]


Validation Accuracy is 66.06


100%|██████████| 1/1 [04:48<00:00, 288.11s/it]


Validation Accuracy is 67.71


100%|██████████| 1/1 [03:23<00:00, 203.40s/it]

Validation Accuracy is 67.89
 Validation Accuracy is 68.0284


In [28]:
df_test[['comment_id', 'score']].to_csv("submission.csv", index=False)